In [1]:
# 基于tensorflow的NiN的实现
# NiN 模型
# NiN块主要由一个卷积层和两个1*1的卷积层组成，第一个卷积层的超参数可以自行确定，后两个卷积层的超参数一般是固定
import numpy as np
import tensorflow as tf 
import random

In [2]:
# 使用GPU进行训练
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

In [3]:
def NiN_block(channels, kernel_size, strides, padding):
    blk = tf.keras.models.Sequential()
    blk.add(tf.keras.layers.Conv2D(filters=channels, kernel_size=kernel_size, 
            strides=strides, padding=padding, activation='relu'))
    blk.add(tf.keras.layers.Conv2D(filters=channels, kernel_size=1, activation='relu'))
    blk.add(tf.keras.layers.Conv2D(filters=channels, kernel_size=1, activation='relu'))
    return blk

In [4]:
# NiN模型
NiN_net = tf.keras.models.Sequential()
NiN_net.add(NiN_block(96, kernel_size=11, strides=4, padding='valid'))
NiN_net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
NiN_net.add(NiN_block(256, kernel_size=5, strides=1, padding='same'))
NiN_net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
NiN_net.add(NiN_block(384, kernel_size=3, strides=1, padding='same'))
NiN_net.add(tf.keras.layers.MaxPool2D(pool_size=3, strides=2))
NiN_net.add(tf.keras.layers.Dropout(0.5))
# 这一层的通道数和labels的类别数相同
NiN_net.add(NiN_block(10, kernel_size=3, strides=1, padding='same'))
# 全局平均池化层
NiN_net.add(tf.keras.layers.GlobalAveragePooling2D())
NiN_net.add(tf.keras.layers.Flatten())

In [5]:
X = tf.random.uniform((1,244,244,1))
for layer in NiN_net.layers:
    X = layer(X)
    print(layer.name, 'output shape:\t', X.shape)

sequential_1 output shape:	 (1, 59, 59, 96)
max_pooling2d output shape:	 (1, 29, 29, 96)
sequential_2 output shape:	 (1, 29, 29, 256)
max_pooling2d_1 output shape:	 (1, 14, 14, 256)
sequential_3 output shape:	 (1, 14, 14, 384)
max_pooling2d_2 output shape:	 (1, 6, 6, 384)
dropout output shape:	 (1, 6, 6, 384)
sequential_4 output shape:	 (1, 6, 6, 10)
global_average_pooling2d output shape:	 (1, 10)
flatten output shape:	 (1, 10)


In [6]:
# 数据处理阶段，同AlexNet 使用fashion_mnist 将28 * 28 扩充为244*244
class DataLoad():
    def __init__(self):
        fashion_mnist = tf.keras.datasets.fashion_mnist
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32) / 255.0, axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32) / 255.0, axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.train_num, self.test_num = self.train_images.shape[0], self.test_images.shape[0]
    
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, self.train_num, batch_size)
        # 填充数据集变为244*244 tf.image
        train_batch = tf.image.resize_with_pad(self.train_images[index], 244, 244)
        return train_batch, self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, self.test_num, batch_size)
        # 填充数据集变为244*244 tf.image
        test_batch = tf.image.resize_with_pad(self.test_images[index], 244, 244)
        return test_batch, self.test_labels[index]

In [7]:
batch_size = 128

data_load  = DataLoad()

In [8]:
# 模型训练阶段
def train_NiN():
    epochs = 5
    num_iter = data_load.train_num // batch_size
    for _ in range(epochs):
        for t in range(num_iter):
            X_batch, Y_batch = data_load.get_batch_train(batch_size)
            NiN_net.fit(X_batch, Y_batch)
            if t % 20 == 0:
                NiN_net.save_weights('NiN_net_weights.h5')
# 优化器
#optimizer = tf.keras.optimizers.SGD(learning_rate=0.06, momentum=0.1, nesterov=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
NiN_net.compile(optimizer=optimizer,
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
train_NiN()

8/8 [==============================] - 0s 45ms/step - loss: 0.5657 - accuracy: 0.8281
